## This Notebook contains the code to Forecast the weekly sales at a Department-Store Level for all 99 departments belonging to 45 stores. 
This is achieved  by training Gated Recurrent Unit (GRU), Long Short Term Memory (LSTM) & Backpropagation Through Time Recurrent Neural Network (BPTT RNN) models.

In [1]:
from __future__ import print_function
import pandas as pd
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
##from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import math
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

*Reading Dept-Store level data*

In [3]:
dept_store_lvl_df = pd.read_csv('data/sales_forecasting.csv')

In [4]:
dept_store_lvl_df.head()

,Store,Dept,Date,Weekly_Sales,Size,Type_A,Type_B,Type_C,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday
0,1,1,2/5/2010,24924.50,151315,1,0,0,42.31,2.572,211.096358,8.106,0
1,1,1,2/12/2010,46039.49,151315,1,0,0,38.51,2.548,211.242170,8.106,1
2,1,1,2/19/2010,41595.55,151315,1,0,0,39.93,2.514,211.289143,8.106,0
3,1,1,2/26/2010,19403.54,151315,1,0,0,46.63,2.561,211.319643,8.106,0
4,1,1,3/5/2010,21827.90,151315,1,0,0,46.50,2.625,211.350143,8.106,0


In [5]:
dept_store_lvl_df.shape

(421570, 13)

#### Creating week_nbr column

In [6]:
dept_store_lvl_df['Date'] = pd.to_datetime(dept_store_lvl_df.Date)
dept_store_lvl_df['week_nbr'] = dept_store_lvl_df.sort_values(['Date'],ascending=[True]).groupby(['Store', 'Dept'])\
             .cumcount() + 1
print(dept_store_lvl_df.head())
print(dept_store_lvl_df.shape)

   Store  Dept       Date  Weekly_Sales    Size  Type_A  Type_B  Type_C  \
0      1     1 2010-02-05      24924.50  151315       1       0       0   
1      1     1 2010-02-12      46039.49  151315       1       0       0   
2      1     1 2010-02-19      41595.55  151315       1       0       0   
3      1     1 2010-02-26      19403.54  151315       1       0       0   
4      1     1 2010-03-05      21827.90  151315       1       0       0   

   Temperature  Fuel_Price         CPI  Unemployment  IsHoliday  week_nbr  
0        42.31       2.572  211.096358         8.106          0         1  
1        38.51       2.548  211.242170         8.106          1         2  
2        39.93       2.514  211.289143         8.106          0         3  
3        46.63       2.561  211.319643         8.106          0         4  
4        46.50       2.625  211.350143         8.106          0         5  
(421570, 14)


*Creating a periodic week_nbr column to capture week of the year value between 1 & 52. This column can capture the seasonality by holding the week of the year value*

In [7]:
a = []
for el in dept_store_lvl_df.week_nbr:
    if el>52 and el<=104:
        a.append(el-52)
    elif el>104:
        a.append(el-104)
    else:
        a.append(el)
week_nbr_periodic = pd.Series(a)
cols = list(dept_store_lvl_df.columns)
cols.append('week_nbr_periodic')
dept_store_lvl_df = pd.concat([dept_store_lvl_df, week_nbr_periodic], axis = 1)
dept_store_lvl_df.columns = cols

In [9]:
### Vary from Weeks 1 to 52 & repeat
print(dept_store_lvl_df.columns)
print(dept_store_lvl_df.week_nbr_periodic.unique())

Index(['Store', 'Dept', 'Date', 'Weekly_Sales', 'Size', 'Type_A', 'Type_B',
       'Type_C', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
       'IsHoliday', 'week_nbr', 'week_nbr_periodic'],
      dtype='object')
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52]


In [11]:
print(dept_store_lvl_df.shape)
dept_store_lvl_df.head()

(421570, 15)


,Store,Dept,Date,Weekly_Sales,Size,Type_A,Type_B,Type_C,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,week_nbr,week_nbr_periodic
0,1,1,2010-02-05,24924.50,151315,1,0,0,42.31,2.572,211.096358,8.106,0,1,1
1,1,1,2010-02-12,46039.49,151315,1,0,0,38.51,2.548,211.242170,8.106,1,2,2
2,1,1,2010-02-19,41595.55,151315,1,0,0,39.93,2.514,211.289143,8.106,0,3,3
3,1,1,2010-02-26,19403.54,151315,1,0,0,46.63,2.561,211.319643,8.106,0,4,4
4,1,1,2010-03-05,21827.90,151315,1,0,0,46.50,2.625,211.350143,8.106,0,5,5


#### Creating a single unified Store Level weekly sales dataset with 10 lagged moving window for every store
Sliding window sequences are created seperately for every store as the lags for every store have to be provided in isolation. Because we don't want to feed in the past values of one store as inputs to another store

In [13]:
((421570+8)/143)

2948.097902097902

In [ ]:
%%capture
stores_dict_ips = {}
stores_dict_tgt3 = {}
stores_dict_df3 = {}
stores_dict3 = {}   ### Creating a dictionary containing the 45 stores

store_train_x = store_lvl_df.iloc[:4433,].drop(columns=['Date','Store','week_nbr'])
store_train_y = store_lvl_df.iloc[:4433,]['Weekly_Sales_sum'].to_frame()
store_test_x = store_lvl_df.iloc[4433:,].drop(columns=['Date','Store','week_nbr'])
store_test_y = store_lvl_df.iloc[4433:,]['Weekly_Sales_sum'].to_frame()

#### Applying Scaler on entire Store Level Aggregated dataset
scaler = MinMaxScaler()
scaler = scaler.fit(store_train_x)
train_arr_x = scaler.transform(store_train_x)
test_arr_x = scaler.transform(store_test_x)
    
scaler = scaler.fit(store_train_y)
train_arr_y = scaler.transform(store_train_y)
test_arr_y = scaler.transform(store_test_y)

#### Creating Sliding Windows for Store 1
x_train, y_train = data_prep(train_arr_x[0:143],train_arr_y[0:143],10)
x_test, y_test = data_prep(test_arr_x[0:143],test_arr_y[0:143],10)

### Creating Sliding windows for other stores 2 to 45 and concatenating them into a single numpy array
for i in range(143,len(train_arr_x),143):   
    ### Calling Data Prep Function to create multivariate sequences (10 lag moving window) Processed
    x_train_store, y_train_store = data_prep(train_arr_x[i:i+143],train_arr_y[i:i+143],10)
    x_train = np.concatenate((x_train, x_train_store), axis=0)
    y_train = np.concatenate((y_train, y_train_store), axis=0)
    
for j in range(143,len(test_arr_x),143):  
    x_test_store, y_test_store = data_prep(test_arr_x[j:j+143],test_arr_y[j:j+143],10)
    x_test = np.concatenate((x_test, x_test_store), axis=0)
    y_test = np.concatenate((y_test, y_test_store), axis=0)
    
### Creating Train & Test Loader Objects
train_target3 = torch.tensor(y_train.astype(np.float32))
train3 = torch.tensor(x_train.astype(np.float32)) 
train_tensor3 = torch.utils.data.TensorDataset(train3, train_target3) 
train_loader3 = torch.utils.data.DataLoader(dataset = train_tensor3, batch_size = len(train3), shuffle = False)
    
test_target3 = torch.tensor(y_test.astype(np.float32))
test3 = torch.tensor(x_test.astype(np.float32)) 
test_tensor3 = torch.utils.data.TensorDataset(test3, test_target3) 
test_loader3 = torch.utils.data.DataLoader(dataset = test_tensor3, batch_size = len(test3),shuffle = False)  